In [1]:
import os

In [2]:
# get the data
output_path = "data"

#!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 
#!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

In [3]:
from hparams import hparams as hps
from src.dataset import load_samples
from src.dataset.dataset import TTSDataset
from src.utils.text import text_to_sequence
from src.models.tacotron2 import Tacotron2

from torch.utils.data import DataLoader


In [4]:

metadata_train, metadata_eval = load_samples(
    'data', 'metadata.csv', True, 50
)

In [5]:
metadata_eval[0]

{'text': 'A molecular change takes place in the nerve of the tentacle,',
 'audio_file': 'data/wavs/LJ025-0134.wav',
 'root_path': 'data'}

In [6]:
text_to_sequence('world!0', cleaner_names=['english_cleaners'])

[60, 52, 55, 49, 41, 2, 63, 42, 55, 52]

In [7]:
eval_ds = TTSDataset(metadata_eval, )

In [8]:
print(eval_ds[1]['mel'].shape)
print(eval_ds[1]['wav'].shape)

(80, 598)
(152989,)


/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils/audio.py:93: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hps.sample_rate, n_fft, n_mels=hps.num_mels, fmin = hps.fmin, fmax = hps.fmax)


In [9]:
print(eval_ds[1]['token_ids'])

[57 45 55 52 58 44 45 11 38 41 59 38 51 40 42 56 11 50 38 41 42 11 57 52
 11 59 38 55 46 52 58 56 11 39 58 46 49 41 42 55 56  6 11 38 51 41 11 57
 45 38 57 11 46 57 11 40 52 58 49 41 11 52 51 49 62 11 50 38 46 51 57 38
 46 51 11 46 57 56 11 40 55 42 41 46 57 11 39 62 11 60 45 52 49 42 56 38
 49 42 11 41 46 56 40 52 58 51 57 46 51 44  7]


In [10]:
eval_loader = DataLoader(
    eval_ds,
    batch_size=hps.batch_size,
    shuffle=False,  # if there is no other sampler
    collate_fn=eval_ds.collate_fn,
    drop_last=False,  # setting this False might cause issues in AMP training.
    num_workers=hps.num_loader_workers,
    pin_memory=False,
)

In [11]:
for batch in eval_loader:
    break

/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils/audio.py:93: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hps.sample_rate, n_fft, n_mels=hps.num_mels, fmin = hps.fmin, fmax = hps.fmax)
/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils/audio.py:93: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hps.sample_rate, n_fft, n_mels=hps.num_mels, fmin = hps.fmin, fmax = hps.fmax)
/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils/audio.py:93: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hps.sample_rate, n_fft, n_mels=hps.num_mels, fmin = hps.fmin, fmax = hps.fmax)
/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils

In [12]:
batch['mel'].shape

torch.Size([4, 598, 80])

In [13]:
model = Tacotron2(hps)

In [14]:
batch['token_ids'].shape

torch.Size([4, 111])

In [15]:
out = model(batch['token_ids'], batch['token_ids_lengths'])

In [19]:
out[0].shape

torch.Size([4, 99, 1024])